In [3]:
# Descriptive Statistics
print("Training Data Descriptive Statistics:")
print(X_agg_train.describe())

print("\nTesting Data Descriptive Statistics:")
print(X_agg_test.describe())


NameError: name 'plt' is not defined

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

# Correlation Matrix
corr_train = X_agg_train.corr()
corr_test = X_agg_test.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_train, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix for Training Data')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_test, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix for Testing Data')
plt.show()


SpecificationError: nested renamer is not supported

In [ ]:
# Distribution of features
for column in X_agg_train.columns:
    plt.figure(figsize=(7, 4))
    sns.histplot(X_agg_train[column], kde=True, color='blue', label='Train')
    sns.histplot(X_agg_test[column], kde=True, color='red', label='Test', alpha=0.6)
    plt.title(f'Distribution of {column}')
    plt.legend()
    plt.show()


In [ ]:
# Missing data analysis
print("Missing Data in Training Dataset:")
print(X_agg_train.isnull().sum())

print("\nMissing Data in Testing Dataset:")
print(X_agg_test.isnull().sum())


In [ ]:
import pandas as pd

def train_test_split_adapted(engine):
    # Ensure 'Date' is in datetime format
    engine.df_concatenated['Date'] = pd.to_datetime(engine.df_concatenated['Date'])
    
    # Sort data by date to ensure chronological order
    engine.df_concatenated.sort_values('Date', inplace=True)
    
    # Determine the cutoff date at two-thirds of the dataset
    unique_dates = engine.df_concatenated['Date'].unique()
    cutoff_index = int(len(unique_dates) * 0.67)
    cutoff_date = unique_dates[cutoff_index]

    # Split the data based on the cutoff date
    engine.X_train = engine.df_concatenated[engine.df_concatenated['Date'] <= cutoff_date].drop(columns=['Type'])
    engine.X_test = engine.df_concatenated[engine.df_concatenated['Date'] > cutoff_date].drop(columns=['Type'])
    engine.y_train = engine.df_concatenated[engine.df_concatenated['Date'] <= cutoff_date]['Type']
    engine.y_test = engine.df_concatenated[engine.df_concatenated['Date'] > cutoff_date]['Type']
    
    # Reset index for both training and testing sets
    engine.X_train.reset_index(drop=True, inplace=True)
    engine.X_test.reset_index(drop=True, inplace=True)
    engine.y_train.reset_index(drop=True, inplace=True)
    engine.y_test.reset_index(drop=True, inplace=True)

    return engine.X_train, engine.X_test, engine.y_train, engine.y_test


In [ ]:
import pandas as pd

# Assuming X_train and X_test are already loaded as pandas DataFrames

# Calculate the mean of each column in the training data
train_means = X_train.mean()

# Fill missing values in X_train with the mean of each column
X_train = X_train.fillna(train_means)

# It's often a good practice to fill missing values in the test set
# using means calculated from the training set to avoid data leakage
X_test = X_test.fillna(train_means)

# Optionally, you can print the datasets to verify that the missing values have been filled
print(X_train.head())
print(X_test.head())


In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Créer des données synthétiques
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

def objective(trial):
    # Définir les hyperparamètres à optimiser
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    # Créer le modèle RandomForest avec les hyperparamètres proposés
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=42
    )

    # Entraîner le modèle
    clf.fit(X_train, y_train)

    # Évaluer le modèle
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

# Créer un environnement Optuna et commencer l'optimisation
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Afficher les meilleurs hyperparamètres
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
